# BERT Fine-Tuned Notebook
## W266 Final Project
### Game of Thrones Text Classification
### T. P. Goter
### Fall 2019

This notebook is used to perform the baseline, finetuned BERT supervised text classification. The original UDA process utilized a Python script wrapped in a bash shell script. This notebook was generated in order to better show and annotate the process.

## Acknowledgement
Much of this code was leveraged from the open source [UDA](https://github.com/google-research/uda). It has been adapted to the Game of Thrones dataset. 

## Import Data Libraries

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import json
import os
import collections
import tensorflow as tf
#tf.enable_eager_execution()
print(tf.__version__)

import yaml
import pprint

from absl import app
from absl import logging

from bert import modeling
from utils import proc_data_utils, raw_data_utils
import uda



/Users/tom/Desktop/MIDS_TPG/W266/nlp_finalproject/tf114/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/tom/Desktop/MIDS_TPG/W266/nlp_finalproject/tf114/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/tom/Desktop/MIDS_TPG/W266/nlp_finalproject/tf114/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16"

1.14.0



/Users/tom/Desktop/MIDS_TPG/W266/nlp_finalproject/tf114/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/tom/Desktop/MIDS_TPG/W266/nlp_finalproject/tf114/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/tom/Desktop/MIDS_TPG/W266/nlp_finalproject/tf114/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 

## Get BERT onto Google Drive - DO THIS ONCE


In [2]:
#! mkdir /content/drive/My\ Drive/NLP_FinalProject/working/bert_pretrained
# download bert base
#! wget -P /content/drive/My\ Drive/NLP_FinalProject/working/bert_pretrained https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
#! unzip /content/drive/My\ Drive/NLP_FinalProject/working/bert_pretrained/uncased_L-12_H-768_A-12.zip &&
#! rm /content/drive/My\ Drive/NLP_FinalProject/working/bert_pretrained/uncased_L-12_H-768_A-12.zip
#! mkdir /content/drive/My\ Drive/NLP_FinalProject/working/bert_pretrained/bert_base
#! mv uncased_L-12_H-768_A-12 /content/drive/My\ Drive/NLP_FinalProject/working/bert_pretrained/bert_base
#! mv /content/drive/My\ Drive/NLP_FinalProject/working/bert_pretrained/bert_base/uncased*/* /content/drive/My\ Drive/NLP_FinalProject/working/bert_pretrained/bert_base/
os.listdir('./bert_pretrained/bert_base/')

['bert_model.ckpt.meta',
 'bert_model.ckpt.data-00000-of-00001',
 'bert_config.json',
 'vocab.txt',
 '.ipynb_checkpoints',
 'bert_model.ckpt.index']

## Define Some Options
This section replaces passing the input parameters as command line arguments. This section is very important. It controls the entire model. See the dictionary below.

### Task Options:
- **do_train:** Boolean of whether we are training
- **do_eval:** Boolean of whether we are just evaluating

### Training Options:
- **sup_train_data_dir:** Input directory for supervised data. This should be set to "./Data/proc_data/train_##" where the ## is one of the subsets of training data generated from the prepro_ALL.csh script.
- **eval_data_dir:**  The input data dir of the evaluation data. This should be the path to the development data with which we will do hyperparameter tuning. We can change this to the test data directory once we are ready for final evaluation. The dev data path is: "./Data/proc_data/dev"
- **unsup_data_dir:** The input data dir of the unsupervised data. Path for the unsupervised, augmented data. This should be equal to "./Data/proc_data/unsup"
- **bert_config_file:** Absolute path to the json file corresponding to the pre-trained BERT model. For us this is: "./bert_pretrained/bert_base/bert_config.json"
- **vocab_file:** The vocabulary file that the BERT model was trained on. This should be equal to "./bert_pretrained/bert_base/vocab.txt"
- **init_checkpoint:** Initial checkpoint from the pre-trained BERT model. This should be equal to: "./bert_pretrained/bert_base/bert_model.ckpt"
- **task_name:** The name of the task to train. This should be equal to "GoT"
- **model_dir:** The output directory where the model checkpoints will be written. This will be set to "models" followed by a case specific identifier.

### Model configuration
- **use_one_hot_embeddings:** Boolean, default: True, If True, tf.one_hot will be used for embedding lookups, otherwise tf.nn.embedding_lookup will be used. On TPUs, this should be True since it is much faster."
- **max_seq_length":** Integer, default = 128, The maximum total sequence length after WordPiece tokenization. Sequences longer than this will be truncated, and sequences shorter than this will be padded. Note, GoT data was processed to be on-average close to this length to minimize lost data.
- **model_dropout:** Float, default = -1 (i.e., no dropout). Dropout rate for both the attention and the hidden states.

### Training hyper-parameters
- **train_batch_size:** Integer, default = 32. Based on the discussion [here](https://github.com/google-research/bert#out-of-memory-issues). 32 is probably the largest we can run with 11 GB of RAM while using BERT base with a maximum sequence length of 128.
- **eval_batch_size:** Integer, default = 8, "Base batch size for evaluation."
- **save_checkpoints_num:** Integer, default = 20, Number of checkpoints to save during training.
- **iterations_per_loop:** Integer, default = 200, Number of steps to make in each estimator call.
- **num_train_steps:** Integer, no default, number of training steps

### Optimizer hyperparameters
- **learning_rate:** Float, default = 2e-5, The initial learning rate for Adam Optimizer
- **num_warmup_steps:** Integer, no default, Number of warmup steps
- **clip_norm:** Float, default= 1.0, Gradient clip hyperparameter.

### UDA Options:
- **unsup_ratio:** Integer - ratio between unsupervised batch size and supervised batch size. If zero - dont use
- **aug_ops:** String - what augmentation procedure do you want to run
- **aug_copy:** Integer - how many augmentations per example are to be generated
- **uda_coeff:** Float - default 1 - This is the coefficient on the UDA loss. Basically you can rely more or less on the UDA loss during the supervised training. The UDA paper generally kept this at 1
- **tsa:** String - Annealing schedule to use. Options provided are "" none, linear_schedule, log_schedule, exp_schedule
- **uda_softmax_temp:** Float, default -1, A smaller temperature will accentuate differences in probabilities. Low temps were used in the UDA paper for cases with low numbers of labeled data, after masking out uncertain predictions.
- **uda_confidence_thresh:** Float, default -1, Threshold value above which the consistency loss term from the UDA is used. Basically ensures we are using loss from random guesses.

### TPU and GPU Options:
- **use_tpu:** Boolean - self-explanatory - it affects how the model is run. If we run in colab this could be important. False means use CPU or GPU. We will default to FALSE.
- **tpu_name:** String - address of the tpu
- **gcp_project:** String - project name when using TPU
- **tpu_zone:** String - can be set or detected
- **master:** Address of the TPU master, if applicable



### Defaults

The defaults below should not be changed. Note that a config file will be read in after this in order to update these if desired.

In [3]:
with open('./config/defaults.yml', 'r') as def_config:
    options = yaml.safe_load(def_config)


## Set the Case to Run
This will ensure that different configurations are being controlled and saved separately. Just load in the correct yaml file that specifies all of the parameters.

### Updated to include UDA parameters

In [4]:
case_specifics = {'do_train': True,
'do_eval' : True,
'sup_train_data_dir': './Data/proc_data/GoT/train_5000',
'eval_data_dir':  './Data/proc_data/GoT/dev',
'bert_config_file':  './bert_pretrained/bert_base/bert_config.json',
'vocab_file':  './bert_pretrained/bert_base/vocab.txt',
'init_checkpoint':  './bert_pretrained/bert_base/bert_model.ckpt',
'task_name':  'GoT',
'model_dir':  './model/shuffle_5000_bs32_hdo15_final',
'num_train_steps':  3000, 
'train_batch_size': 32,
'learning_rate':  0.00003,
'num_warmup_steps':  300, 
'hidden_dropout': 0.15,
'attention_dropout': 0.1,
'save_checkpoints_num' :10,
'use_tpu' :False,
'use_one_hot_embeddings': False   }

In [5]:
# merge dictionaries    
options.update(case_specifics)

#
print()
print("="*50 + "\nFull Listing of Options: \n" + "="*50)
pprint.pprint(options)


Full Listing of Options: 
{'attention_dropout': 0.1,
 'aug_copy': -1,
 'aug_ops': '',
 'bert_config_file': './bert_pretrained/bert_base/bert_config.json',
 'clip_norm': 1.0,
 'do_eval': True,
 'do_train': True,
 'eval_batch_size': 8,
 'eval_data_dir': './Data/proc_data/GoT/dev',
 'hidden_dropout': 0.15,
 'init_checkpoint': './bert_pretrained/bert_base/bert_model.ckpt',
 'iterations_per_loop': 200,
 'learning_rate': 3e-05,
 'max_seq_length': 128,
 'model_dir': './model/shuffle_5000_bs32_hdo15_final',
 'num_train_steps': 3000,
 'num_warmup_steps': 300,
 'save_checkpoints_num': 10,
 'sup_train_data_dir': './Data/proc_data/GoT/train_5000',
 'task_name': 'GoT',
 'train_batch_size': 32,
 'tsa': '',
 'uda_coeff': 1,
 'uda_confidence_thresh': -1,
 'uda_softmax_temp': -1,
 'unsup_data_dir': 'None',
 'unsup_ratio': 0,
 'use_one_hot_embeddings': False,
 'use_tpu': False,
 'vocab_file': './bert_pretrained/bert_base/vocab.txt'}


## Setup the Job
This section of the code grabs the right data and reads in the BERT config file. We also dump our configuration options to a JSON file in the model directory.

In [6]:
# Record informational logs
logging.set_verbosity(logging.INFO)

# GoT Processor Class has methods for retrieving text/train/dev datasets
processor = raw_data_utils.GoTProcessor()

# Read in the labels
label_list = processor.get_labels()

# Check the labels  -  they should be 1 through 5
print(label_list)

# Read the BertConfig File
bert_config = modeling.BertConfig.from_json_file(
      options['bert_config_file'],
      options['hidden_dropout'],
      options['attention_dropout'])

# Create the directory for the current model
tf.io.gfile.makedirs(options['model_dir'])

# Dump the configuration dictionary to an output json file in the model specific directory
tf.io.write_file(os.path.join(options['model_dir'], "OPTIONS.json"), json.dumps(options))

['1', '2', '3', '4', '5']

INFO:tensorflow:Setting up BERT Config using data from ./bert_pretrained/bert_base/bert_config.json
INFO:tensorflow:Setting up BERT Config using data from {'attention_probs_dropout_prob': 0.1, 'hidden_act': 'gelu', 'hidden_dropout_prob': 0.1, 'hidden_size': 768, 'initializer_range': 0.02, 'intermediate_size': 3072, 'max_position_embeddings': 512, 'num_attention_heads': 12, 'num_hidden_layers': 12, 'type_vocab_size': 2, 'vocab_size': 30522}
INFO:tensorflow:Updating FF layer dropout to 0.15
INFO:tensorflow:Updating attention layer dropout to 0.1


<tf.Operation 'WriteFile' type=WriteFile>

In [7]:
if options['use_tpu'] and tpu_address:
  tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(
      tpu_address, zone=None, project=None)
else:
  tpu_cluster_resolver = None

## Model Specific Setup

In [8]:
logging.info("warmup steps {}/{}".format(
      options['num_warmup_steps'], options['num_train_steps']))

# Specify where the checkpoints will be saved. This is just integer division between the total number of training steps and the number of checkpoints
save_checkpoints_steps = options['num_train_steps'] // options['save_checkpoints_num']

# Log the checkpoints
logging.info("setting save checkpoints steps to {:d}".format(
      save_checkpoints_steps))

# Update iterations per loop
options['iterations_per_loop'] = min(save_checkpoints_steps,
                                  options['iterations_per_loop'])

INFO:absl:warmup steps 300/3000
INFO:absl:setting save checkpoints steps to 300


## Setup Run Configuration

In [9]:
# This is set up for potential use with a TPU
is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2
run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    master=None,
    model_dir=options['model_dir'],
    save_checkpoints_steps=save_checkpoints_steps,
    keep_checkpoint_max=10,
    # train_distribute=train_distribute,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=options['iterations_per_loop'],
        per_host_input_for_training=is_per_host))


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## Create our model
Our model uses the bert configuration parameters as well as gets fed all of our particular running options.

In [10]:
model_fn = uda.model_fn_builder(
      bert_config=bert_config,
      init_checkpoint=options['init_checkpoint'],
      learning_rate=options['learning_rate'],
      clip_norm=options['clip_norm'],
      num_train_steps=options['num_train_steps'],
      num_warmup_steps=options['num_warmup_steps'],
      use_tpu=options['use_tpu'],
      use_one_hot_embeddings=options['use_one_hot_embeddings'],
      num_labels=len(label_list),
      unsup_ratio=options['unsup_ratio'],
      uda_coeff=options['uda_coeff'],
      tsa=options['tsa'],
      uda_softmax_temp=options['uda_softmax_temp'],
      uda_confidence_thresh=options['uda_confidence_thresh'],
      print_feature=False,
      print_structure=False,
      freeze_layers = (False, )
  )

## Create our Estimator
Basically contains the intstructions to run the model function.

In [11]:

estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=options['use_tpu'],
    model_fn=model_fn,
    config=run_config,
    params={"model_dir": options['model_dir']},
    train_batch_size=options['train_batch_size'],
    eval_batch_size=options['eval_batch_size'],
    predict_batch_size=options['eval_batch_size'])



INFO:tensorflow:Using config: {'_model_dir': './model/shuffle_5000_bs32_hdo15_final', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 300, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 10, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x133f3c438>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=200, num_shards=None, num_cores_per_replica=None, per_host_input_for_

INFO:tensorflow:Using config: {'_model_dir': './model/shuffle_5000_bs32_hdo15_final', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 300, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 10, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x133f3c438>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=200, num_shards=None, num_cores_per_replica=None, per_host_input_for_

INFO:tensorflow:_TPUContext: eval_on_tpu True


INFO:tensorflow:_TPUContext: eval_on_tpu True


## Ready to Train

In [12]:
# Logical check to determine if we are training (vice evaluating)
if options['do_train']:
    logging.info("Getting data for supervised training from: {}".format(options['sup_train_data_dir']))
    
    # Are we doing UDA or just simple finetuning?
    if options['unsup_ratio'] > 0:
        logging.info("Getting unsupervised data from: {}".format(
          options['unsup_data_dir']))
    
    # Pass on all of the training sup/unsup options
    train_input_fn = proc_data_utils.training_input_fn_builder(
        options['sup_train_data_dir'],
        max_seq_len=options['max_seq_length'],
        unsup_data_base_path=options['unsup_data_dir'],
        aug_ops=options['aug_ops'],
        aug_copy=options['aug_copy'],
        unsup_ratio=options['unsup_ratio'])

INFO:absl:Getting data for supervised training from: ./Data/proc_data/GoT/train_5000


INFO:tensorflow:loading training data from these files: ./Data/proc_data/GoT/train_5000/tf_examples.tfrecord.0.1 ./Data/proc_data/GoT/train_5000/tf_examples.tfrecord.0.0


INFO:tensorflow:loading training data from these files: ./Data/proc_data/GoT/train_5000/tf_examples.tfrecord.0.1 ./Data/proc_data/GoT/train_5000/tf_examples.tfrecord.0.0


In [13]:
# Logical check to see if we are evaluating against the development set (or test set if you change the eval_data_dir)
if options['do_eval']:
    logging.info("  >>> dev data dir : {}".format(options['eval_data_dir']))
    eval_input_fn = proc_data_utils.evaluation_input_fn_builder(
        options['eval_data_dir'],
        "clas",
        max_seq_len=options['max_seq_length'])

    eval_size = processor.get_dev_size()
    eval_steps = int(eval_size / options['eval_batch_size'])
    logging.info("Evaluating {} examples in {} steps".format(eval_size,eval_steps))

INFO:absl:  >>> dev data dir : ./Data/proc_data/GoT/dev


INFO:tensorflow:loading eval clas data from these files: ./Data/proc_data/GoT/dev/tf_examples.tfrecord.0.0


INFO:tensorflow:loading eval clas data from these files: ./Data/proc_data/GoT/dev/tf_examples.tfrecord.0.0
INFO:absl:Evaluating 2500 examples in 312 steps


In [14]:
# IF we are training and evaluating
if options['do_train'] and options['do_eval']:
    logging.info("***** Running training & evaluation *****")
    logging.info("  Supervised batch size = {:d}".format(
        options['train_batch_size']))
    logging.info("  Unsupervised batch size = {:d}".format(
        options['train_batch_size'] * options['unsup_ratio']))
    logging.info("  Num steps = {}".format(options['num_train_steps']))
    logging.info("  Base evaluation batch size = {:d}".format(
        options['eval_batch_size']))
    logging.info("  Num steps = {:d}".format(eval_steps))
    
    # Initialize
    best_acc = 0
    
    # Looping over training steps by subset (for each checkpoint)
    for s in range(0, options['num_train_steps'], save_checkpoints_steps):
        logging.info("*** Running training: Current Step: {}***".format(s))
        
        estimator.train(
              input_fn=train_input_fn,
              steps=save_checkpoints_steps)
        
        logging.info("*** Running evaluation ***")
        dev_result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)
        logging.info(">> Results:")
        
        # Keep track of the evaluation results
        for key in dev_result.keys():
            logging.info("  {} = {}".format(key, str(dev_result[key])))
            dev_result[key] = dev_result[key].item()
        
        # Update the best accuracy object
        best_acc = max(best_acc, dev_result["eval_classify_accuracy"])
    logging.info("***** Final evaluation result *****")
    logging.info("Best acc: {:.3f}\n\n".format(best_acc))
elif options['do_train']:
    logging.info("***** Running training *****")
    logging.info("  Supervised batch size = {}".format(options['train_batch_size']))
    logging.info("  Unsupervised batch size = {}".format(
                    options['train_batch_size'] * options['unsup_ratio']))
    logging.info("  Num steps = {}".format(options['num_train_steps']))
    estimator.train(input_fn=train_input_fn, max_steps=options['num_train_steps'])
elif options['do_eval']:
    logging.info("***** Running evaluation *****")
    logging.info("  Base evaluation batch size = {}".format(options['eval_batch_size']))
    logging.info("  Num steps = {}".format(eval_steps))
    
    # Load in the checkpoint from training to do the evaluation
    checkpoint_state = tf.train.get_checkpoint_state(options['model_dir'])

    best_acc = 0
    for ckpt_path in checkpoint_state.all_model_checkpoint_paths:
        if not tf.io.gfile.exists(ckpt_path + ".data-00000-of-00001"):
            logging.info(
                "Warning: checkpoint {:s} does not exist".format(ckpt_path))
        continue
        logging.info("Evaluating {:s}".format(ckpt_path))
        dev_result = estimator.evaluate(
          input_fn=eval_input_fn,
          steps=eval_steps,
          checkpoint_path=ckpt_path,)
        logging.info(">> Results:")
        
        # keep track of evaluation metrics
        for key in dev_result.keys():
            logging.info("  {:s} = {:s}".format(key, str(dev_result[key])))
            dev_result[key] = dev_result[key].item()
        
        # update our best accuracy variable
        best_acc = max(best_acc, dev_result["eval_classify_accuracy"])
    logging.info("***** Final evaluation result *****")
    logging.info("Best acc: {:.3f}\n\n".format(best_acc))

INFO:absl:***** Running training & evaluation *****
INFO:absl:  Supervised batch size = 32
INFO:absl:  Unsupervised batch size = 0
INFO:absl:  Num steps = 3000
INFO:absl:  Base evaluation batch size = 8
INFO:absl:  Num steps = 312
INFO:absl:*** Running training: Current Step: 0***


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


INFO:tensorflow:Supervised batch size: 32


INFO:tensorflow:Supervised batch size: 32


...

INFO:tensorflow:Saving checkpoints for 0 into ./model/shuffle_5000_bs32_hdo15_final/model.ckpt.


INFO:tensorflow:Saving checkpoints for 0 into ./model/shuffle_5000_bs32_hdo15_final/model.ckpt.


INFO:tensorflow:training_loop marked as finished


INFO:tensorflow:training_loop marked as finished


KeyboardInterrupt: 